# Imports 

In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from functools import reduce

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import  accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor


# Data Retrieval

Download Access database and Use SQLlite

Pull in saved CSVs

In [16]:
df_private = pd.read_csv("data/private_financials.csv").iloc[:, 1:]
df_public = pd.read_csv("data/public_financials.csv").iloc[:, 1:]
df_ids = pd.read_csv("data/university_ids.csv").iloc[:,0:2]
df_ids.rename(columns={'IPEDS\nUnit ID': 'UNITID', 'Organization or School Name':'School'}, inplace=True)


FileNotFoundError: [Errno 2] No such file or directory: '/c/Users/andy.schmeck.OFFICE/Documents/Flatiron/college_recommendation/data/private_financials.csv'

In [112]:
private_ids = [df_ids, df_private]
public_ids = [df_ids, df_public]
df_private= reduce(lambda x, y: pd.merge(x, y, on = 'UNITID'), private_ids)
df_public= reduce(lambda x, y: pd.merge(x, y, on = 'UNITID'), public_ids)

In [85]:
df_private.head()

,UNITID,School,F2A01,F2A19,F2A20,F2A02,F2A03,F2A03A,F2A04,F2A05,...,F2E113,F2E114,F2E115,F2E116,F2E117,F2E123,F2E124,F2E125,F2E126,F2E127
0,177834.0,A. T. Still University of Health Sciences,206876335.0,111669363.0,0.0,364820808.0,114720321.0,87304237.0,168113999.0,81986488.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,177834.0,A. T. Still University of Health Sciences,178774070.0,113671486.0,0.0,342501998.0,118019149.0,89158604.0,148820102.0,75662747.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,177834.0,A. T. Still University of Health Sciences,169806733.0,118593869.0,0.0,324145899.0,121502323.0,90988344.0,130349544.0,72294032.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,177834.0,A. T. Still University of Health Sciences,148752196.0,93801916.0,0.0,276602600.0,92284317.0,61375816.0,120238858.0,64079425.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,177834.0,A. T. Still University of Health Sciences,152052341.0,96318825.0,0.0,281678255.0,96171512.0,63174804.0,119718977.0,65787766.0,...,547675.0,-8551373.0,2413118.0,609252.0,3735841.0,0.0,0.0,0.0,0.0,0.0


# Column Inclusion

Many of the financial columns are just the same data in relation to full time employers (FTE)(ending in FT) or in percentage of revenue/expenses (ending in PC). I kept the percentage of budget because allocating money in hand is different than trying to earn more to spend more. 

In [75]:
df_public.corr() > .75

TypeError: corr() missing 1 required positional argument: 'other'

In [99]:
#if you don't have the target, why are you even here?
df_private = df_private[~df_private['GBA6RTBK'].isna()]
#and again for public
df_public = df_public[~df_public['GBA6RTBK'].isna()]

In [100]:
correlates= df_private[df_private.columns[1:]].corr()['GBA6RTBK'] > .5
correlates[correlates == True]

GBA6RTBK    True
F2TUFEFT    True
Name: GBA6RTBK, dtype: bool

In [102]:
correlates= df_public[df_public.columns[1:]].corr()['GBA6RTBK'] > .5
correlates[correlates ==True]

GBA6RTBK    True
Name: GBA6RTBK, dtype: bool

In [113]:
#keep non colinear columns
df_private_trim = df_private[['UNITID','School','F2TUFEFT','F2GVGCFT',
'F2PGGCFT','F2INVRFT','F2OTRVFT','F2COREXP','F2PBSVFT', 'F2INSTFT',
'F2ACSPFT','F2STSVFT','F2OTEXFT', 'F2EQUITR','GBA6RTBK', 
'year']]
df_private_trim.rename(columns={"F2TUFEFT":"Tuition", "F2GVGCFT":"Gov_grants", 
"F2INSTFT": "Instruction", "F2PGGCFT":"Priv_grants", "F2INVRFT":"Investments",
"F2OTRVFT":"Other_rev", "F2COREXP":"Total_expenses", "F2PBSVFT":"Public_services",
"F2STSVFT":"Student_services","F2OTEXFT":"Other_exp","F2ACSPFT":"Academic_support",
"F2EQUITR":"Equity_ratio","GBA6RTBK":"Black_GR"}, inplace=True)

#again for public
df_public_trim = df_public[['UNITID','School','F1TUFEFT',
'F1LCAPFT','F1COREXP','F1PBSVFT', 'F1ACSPFT','F1STSVFT',
'F1OTEXFT','F1EQUITR','GBA6RTBK', 'year',
'F1MHP', 'F1FHA', 'F1E03', 'F1E09']]
df_public_trim.rename(columns={"F1TUFEFT":"Tuition", "F1LCAPFT":"Local_approp", 
"F1COREXP":"Total_expenses", "F1PBSVFT":"Public_services", "F1ACSPFT":"Academic_support", "F1STSVFT":"Student_services",
"F1OTEXFT":"Other_exp","F1EQUITR":"Equity_ratio","GBA6RTBK":"Black_GR",
"F1MHP":"Pension", "F1FHA":"Endowment", "F1E03":"Grants_by_state", "F1E09":"Discounts"}, inplace=True)


C:\Users\andy.schmeck.OFFICE\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [114]:
df_public_trim.corr() > .75

,UNITID,Tuition,Local_approp,Total_expenses,Public_services,Academic_support,Student_services,Other_exp,Equity_ratio,Black_GR,year,Pension,Endowment,Grants_by_state,Discounts
UNITID,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Tuition,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
Local_approp,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
Total_expenses,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
Public_services,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
Academic_support,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
Student_services,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
Other_exp,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
Equity_ratio,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
Black_GR,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


In [66]:
df_public[['F1GVGCFT', 'F1PGGCFT', 'F1INVRFT', 'F1OTRVFT', 'GBA6RTBK']].corr() > .75


,F1GVGCFT,F1PGGCFT,F1INVRFT,F1OTRVFT,GBA6RTBK
F1GVGCFT,True,False,False,True,False
F1PGGCFT,False,True,False,False,False
F1INVRFT,False,False,True,False,False
F1OTRVFT,True,False,False,True,False
GBA6RTBK,False,False,False,False,True


In [67]:
df_public[['F1INSTFT', 'F1RSRCFT', 'F1INSUFT', 'F1ENDMFT', 'GBA6RTBK']].corr() > .75


,F1INSTFT,F1RSRCFT,F1INSUFT,F1ENDMFT,GBA6RTBK
F1INSTFT,True,False,False,False,False
F1RSRCFT,False,True,False,False,False
F1INSUFT,False,False,True,False,False
F1ENDMFT,False,False,False,True,False
GBA6RTBK,False,False,False,False,True


In [68]:
df_private[['F2RSRCFT', 'F2ENDMFT', 'GBA6RTBK']].corr() > .75


,F2RSRCFT,F2ENDMFT,GBA6RTBK
F2RSRCFT,True,True,False
F2ENDMFT,True,True,False
GBA6RTBK,False,False,True


In [69]:
df_private[['F2ACSPFT', 'F2STSVFT', 'GBA6RTBK']].corr() > .75


,F2ACSPFT,F2STSVFT,GBA6RTBK
F2ACSPFT,True,True,False
F2STSVFT,True,True,False
GBA6RTBK,False,False,True


# Data Cleaning

In [105]:
#if you don't have the target, why are you even here?
df_private = df_private[~df_private['GBA6RTBK'].isna()]
#and again for public
df_public = df_public[~df_public['GBA6RTBK'].isna()]

In [106]:
#drop rows with no information
df_private.set_index(['UNITID', 'year'], inplace=True)
df_private.dropna(how='all')


,,F2TUFEPC,F2GVGCPC,F2PGGCPC,F2INVRPC,F2OTRVPC,F2TUFEFT,F2COREXP,F2INSTPC,F2RSRCPC,F2PBSVPC,...,F2RSRCFT,F2PBSVFT,F2ACSPFT,F2STSVFT,F2INSUFT,F2OTEXFT,F2SAFBPC,F2ENDMFT,F2EQUITR,GBA6RTBK
UNITID,year,,,,,,,,,,,,,,,,,,,,,
100690,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
100937,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0
101189,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
101365,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
101435,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414823,2014,83.0,1.0,2.0,12.0,2.0,15584.0,34246456.0,36.0,0.0,0.0,...,0.0,62.0,2250.0,4211.0,4458.0,0.0,54.0,17288.0,74.0,34.0
433536,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
439288,2014,83.0,1.0,2.0,12.0,2.0,15454.0,25509565.0,36.0,0.0,0.0,...,0.0,61.0,2231.0,4176.0,4421.0,0.0,54.0,17145.0,74.0,13.0


In [107]:
#and again for public
df_public.set_index(['UNITID', 'year'], inplace=True)
df_public.dropna(how='all')

,,F1TUFEPC,F1GVGCPC,F1PGGCPC,F1INVRPC,F1OTRVPC,F1TUFEFT,F1STAPFT,F1LCAPFT,F1COREXP,F1INSTPC,...,F1RSRCFT,F1PBSVFT,F1ACSPFT,F1STSVFT,F1INSUFT,F1OTEXFT,F1SAFBPC,F1ENDMFT,F1EQUITR,GBA6RTBK
UNITID,year,,,,,,,,,,,,,,,,,,,,,
100654,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
100663,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0
100706,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0
100724,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0
100751,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436818,2014,19.0,14.0,3.0,0.0,27.0,2367.0,4604.0,0.0,21600137.0,45.0,...,1422.0,774.0,1490.0,516.0,1314.0,102.0,67.0,2943.0,52.0,43.0
436827,2014,19.0,14.0,3.0,0.0,27.0,7517.0,14623.0,0.0,19058945.0,45.0,...,4516.0,2459.0,4732.0,1640.0,4172.0,323.0,67.0,9347.0,52.0,50.0
436836,2014,19.0,14.0,3.0,0.0,27.0,3126.0,6082.0,0.0,15247156.0,45.0,...,1878.0,1023.0,1968.0,682.0,1735.0,134.0,67.0,3887.0,52.0,43.0


# Feature Engineering

binning correlates

In [108]:
df_public['Rev_not_tuition'] =df_public[['F1GVGCFT'] * df_public['F1PGGCFT'] *df_public['F1INVRFT'] *df_public['F1OTRVFT']
df_public['Instruction_and_research'] =df_public['F1INSTFT']*df_public['F1RSRCFT'] * df_public['F1INSUFT'] *df_public['F1ENDMFT']

df_private['Research_and_endowment']= df_private['F2RSRCFT']*df_private['F2ENDMFT']
df_private['Academic_and_student_support']= df_private['F2ACSPFT']*df_private['F2STSVFT']



In [109]:
# for public
#  df_public['GBA6RTBK']

# Target 

In [110]:
df_private['GBA6RTBK'].describe()

count    6089.000000
mean       43.877320
std        26.834411
min         0.000000
25%        25.000000
50%        42.000000
75%        62.000000
max       100.000000
Name: GBA6RTBK, dtype: float64

### Binary

In [111]:
# #bin target to binary
# df_private.loc[df_private['GBA6RTBK'] > 66,'abv_avg_gr'] = 1
# df_private.loc[df_private['GBA6RTBK'] <= 66,'abv_avg_gr'] = 0
# df_private['abv_avg_gr'].value_counts()

In [112]:
# #and again for public
# df_public.loc[df_public['GBA6RTBK'] > 58,'abv_avg_gr'] = 1
# df_public.loc[df_public['GBA6RTBK'] <= 58,'abv_avg_gr'] = 0
# df_public['abv_avg_gr'].value_counts()

In [113]:
# #private
# y_private = df_private['abv_avg_gr']
# X_private = df_private.drop(columns = ['GBA6RTBK', 'abv_avg_gr'], axis = 1)
# X_train_private, X_test_private, y_train_private, y_test_private = train_test_split(X_private, y_private, random_state = 42)
# #and for public
# y_public = df_public['abv_avg_gr']
# X_public = df_public.drop(columns = ['GBA6RTBK', 'abv_avg_gr'], axis = 1)
# X_train_public, X_test_public, y_train_public, y_test_public = train_test_split(X_public, y_public, random_state = 42)


In [114]:
# #Concatenate to one dataframe, check for nan's
# df_private = pd.concat([X_train_private, y_train_private], axis=1)
# df_private.dropna(inplace=True)
# #for public
# df_public = pd.concat([X_train_public, y_train_public], axis=1)
# df_public.dropna(inplace=True)

In [115]:
# #split back
# y_train_private = df_private['abv_avg_gr']
# X_train_private = df_private.drop(columns = ['abv_avg_gr'], axis = 1)
# #for public
# y_train_public = df_public['abv_avg_gr']
# X_train_public = df_public.drop(columns = ['abv_avg_gr'], axis = 1)

### Ternary
Divide the observations into three equal bins, based on black graduation rates. 

In [116]:
#ternary bining
bin_labels =['low', 'medium', 'high']
df_private['gr_rank'] = pd.qcut(df_private['GBA6RTBK'], q=3, labels=bin_labels)

In [117]:
#for public
df_public['gr_rank'] = pd.qcut(df_public['GBA6RTBK'], q=3, labels=bin_labels)

In [118]:
#private
y_private = df_private['gr_rank']
X_private = df_private.drop(columns = ['GBA6RTBK', 'gr_rank'], axis = 1)
X_train_private, X_test_private, y_train_private, y_test_private = train_test_split(X_private, y_private, random_state = 42)
#and for public
y_public = df_public['gr_rank']
X_public = df_public.drop(columns = ['GBA6RTBK', 'gr_rank'], axis = 1)
X_train_public, X_test_public, y_train_public, y_test_public = train_test_split(X_public, y_public, random_state = 42)


In [119]:
#Concatenate to one dataframe, check for nan's
df_private = pd.concat([X_train_private, y_train_private], axis=1)
df_private.dropna(inplace=True)
#for public
df_public = pd.concat([X_train_public, y_train_public], axis=1)
df_public.dropna(inplace=True)

In [120]:
#split back
y_train_private = df_private['gr_rank']
X_train_private = df_private.drop(columns = ['gr_rank'], axis = 1)
#for public
y_train_public = df_public['gr_rank']
X_train_public = df_public.drop(columns = ['gr_rank'], axis = 1)


### Continuous

In [121]:
# #private
# y_private = df_private['GBA6RTBK']
# X_private = df_private.drop(columns = ['GBA6RTBK'], axis = 1)
# X_train_private, X_test_private, y_train_private, y_test_private = train_test_split(X_private, y_private, random_state = 42)
# #and for public
# y_public = df_public['GBA6RTBK']
# X_public = df_public.drop(columns = ['GBA6RTBK'], axis = 1)
# X_train_public, X_test_public, y_train_public, y_test_public = train_test_split(X_public, y_public, random_state = 42)


In [122]:
# #Concatenate to one dataframe, check for nan's
# df_private = pd.concat([X_train_private, y_train_private], axis=1)
# df_private.dropna(inplace=True)
# #for public
# df_public = pd.concat([X_train_public, y_train_public], axis=1)
# df_public.dropna(inplace=True)

In [123]:
# #split back
# y_train_private = df_private['GBA6RTBK']
# X_train_private = df_private.drop(columns = ['GBA6RTBK'], axis = 1)
# #for public
# y_train_public = df_public['GBA6RTBK']
# X_train_public = df_public.drop(columns = ['GBA6RTBK'], axis = 1)


# First Simple Model

## Pipeline

In [124]:
#Set up pipeline for scaling continuous variables
continuous_pipeline_private= Pipeline(steps=[
    ('ss', StandardScaler())
])

trans_private = ColumnTransformer(transformers=[
    ('continuous', continuous_pipeline_private, X_train_private.columns),
])


In [125]:
#for public
continuous_pipeline_public = Pipeline(steps=[
    ('ss', StandardScaler())
])

trans_public = ColumnTransformer(transformers=[
    ('continuous', continuous_pipeline_public, X_train_public.columns),
])

## Dummy

In [126]:
#Pipeline for running the model
dummy_private = Pipeline(steps=[
    ('trans', trans_private),
    ('dummy', DummyClassifier(random_state = 42))
])
#Fitting and checking the score
dummy_private.fit(X_train_private, y_train_private)
dummy_private.score(X_train_private, y_train_private)


In [127]:
#Public
dummy_public = Pipeline(steps=[
    ('trans', trans_public),
    ('dummy', DummyClassifier(random_state = 42))
])
#Fitting and checking the score
dummy_public.fit(X_train_public, y_train_public)
dummy_public.score(X_train_public, y_train_public)

## Decision Tree

In [128]:
model_one_private = Pipeline(steps=[
    ('trans', trans_private),
    ('simple_dt', DecisionTreeClassifier(max_depth = 5, random_state = 42))
])
#Fit model on all the data
model_one_private.fit(X_train_private, y_train_private)
#Grab predictions and print precision
y_pred_private = model_one_private.predict(X_train_private)
print("Training Score:" + str(accuracy_score(y_train_private, y_pred_private)))
#Run a cross validation to test for overfitting
scores_private = np.mean(cross_val_score(model_one_private, X_train_private, y_train_private, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores_private))

In [129]:
model_one_public = Pipeline(steps=[
    ('trans', trans_public),
    ('simple_dt', DecisionTreeClassifier(max_depth = 5, random_state = 42))
])
#Fit model on all the data
model_one_public.fit(X_train_public, y_train_public)
#Grab predictions and print precision
y_pred_public = model_one_public.predict(X_train_public)
print("Training Score:" + str(accuracy_score(y_train_public, y_pred_public)))
#Run a cross validation to test for overfitting
scores_public = np.mean(cross_val_score(model_one_public, X_train_public, y_train_public, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores_public))

In [130]:
#Feature Importance
important_private = []
for name, importance in zip(X_train_private.columns, model_one_private['simple_dt'].feature_importances_):
    if importance > 0:
        important_private.append((name, importance))

important_private

In [131]:
#For Public
important_public = []
for name, importance in zip(X_train_public.columns, model_one_public['simple_dt'].feature_importances_):
    if importance > 0:
        important_public.append((name, importance))
important_public

## Logistic Regression

In [ ]:
logreg_private = Pipeline(steps=[
    ('trans', trans_private),
    ('logreg', LogisticRegression(multi_class="multinomial"))
])
#Fit model on all the data
logreg_private.fit(X_train_public, y_train_public)
#Grab predictions and print precision
y_pred_public = logreg_private.predict(X_train_public)
print("Training Score:" + str(accuracy_score(y_train_public, y_pred_public)))
#Run a cross validation to test for overfitting
scores_public = np.mean(cross_val_score(logreg_private, X_train_public, y_train_public, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores_public))

In [ ]:
logreg_public = Pipeline(steps=[
    ('trans', trans_private),
    ('logreg', LogisticRegression(multi_class="multinomial"))
])
#Fit model on all the data
logreg_public.fit(X_train_public, y_train_public)
#Grab predictions and print precision
y_pred_public = logreg_public.predict(X_train_public)
print("Training Score:" + str(accuracy_score(y_train_public, y_pred_public)))
#Run a cross validation to test for overfitting
scores_public = np.mean(cross_val_score(logreg_public, X_train_public, y_train_public, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores_public))

# Reduce Features by importance

In [132]:
# import statsmodels.api as sm
# def stepwise_selection(X, y, 
#                        initial_list=[], 
#                        threshold_in=0.01, 
#                        threshold_out = 0.05, 
#                        verbose=True):
#     """ Perform a forward-backward feature selection 
#     based on p-value from statsmodels.api.OLS
#     Arguments:
#         X - pandas.DataFrame with candidate features
#         y - list-like with the target
#         initial_list - list of features to start with (column names of X)
#         threshold_in - include a feature if its p-value < threshold_in
#         threshold_out - exclude a feature if its p-value > threshold_out
#         verbose - whether to print the sequence of inclusions and exclusions
#     Returns: list of selected features 
#     Always set threshold_in < threshold_out to avoid infinite looping.
#     See https://en.wikipedia.org/wiki/Stepwise_regression for the details
#     """
#     included = list(initial_list)
#     while True:
#         changed=False
#         # forward step
#         excluded = list(set(X.columns)-set(included))
#         new_pval = pd.Series(index=excluded)
#         for new_column in excluded:
#             model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
#             new_pval[new_column] = model.pvalues[new_column]
#         best_pval = new_pval.min()
#         if best_pval < threshold_in:
#             best_feature = new_pval.idxmin()
#             included.append(best_feature)
#             changed=True
#             #if verbose:
#                 #print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

#         # backward step
#         model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
#         # use all coefs except intercept
#         pvalues = model.pvalues.iloc[1:]
#         worst_pval = pvalues.max() # null if pvalues is empty
#         if worst_pval > threshold_out:
#             changed=True
#             worst_feature = pvalues.argmax()
#             included.remove(worst_feature)
#             #if verbose:
#                 #print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
#         if not changed:
#             break
#     included.append('id')
#     print('resulting features:')
#     print(included)
    
#     return included

In [133]:
# stepwise_selection(X_train_private,y_train_private)

<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-132-aba68b6436e6>

resulting features:
['F2TUFEFT', 'F2ENDMFT', 'F2INSUPC', 'F2TUFEPC', 'F2OTEXPC', 'F2STSVPC', 'F2STSVFT', 'F2EQUITR', 'id']


['F2TUFEFT',
 'F2ENDMFT',
 'F2INSUPC',
 'F2TUFEPC',
 'F2OTEXPC',
 'F2STSVPC',
 'F2STSVFT',
 'F2EQUITR',
 'id']

In [136]:
# X_train_private = X_train_private[['F2TUFEFT', 'F2ENDMFT', 'F2INSUPC', 'F2TUFEPC', 'F2OTEXPC', 'F2STSVPC', 'F2STSVFT', 'F2EQUITR']]

In [135]:
# stepwise_selection(X_train_public, y_train_public)

<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-132-aba68b6436e6>

resulting features:
['F1TUFEFT', 'F1COREXP', 'F1EQUITR', 'F1GVGCPC', 'F1RSRCPC', 'F1PBSVPC', 'F1STAPFT', 'F1ACSPFT', 'F1ACSPPC', 'F1INSTPC', 'F1TUFEPC', 'F1PGGCPC', 'F1LCAPFT', 'F1OTRVPC', 'id']


<ipython-input-132-aba68b6436e6>:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


['F1TUFEFT',
 'F1COREXP',
 'F1EQUITR',
 'F1GVGCPC',
 'F1RSRCPC',
 'F1PBSVPC',
 'F1STAPFT',
 'F1ACSPFT',
 'F1ACSPPC',
 'F1INSTPC',
 'F1TUFEPC',
 'F1PGGCPC',
 'F1LCAPFT',
 'F1OTRVPC',
 'id']

In [137]:
# X_train_public=X_train_public[['F1TUFEFT', 'F1COREXP', 'F1EQUITR', 'F1GVGCPC', 'F1RSRCPC', 'F1PBSVPC', 'F1STAPFT', 'F1ACSPFT', 'F1ACSPPC', 'F1INSTPC', 'F1TUFEPC', 'F1PGGCPC', 'F1LCAPFT', 'F1OTRVPC']]

# Linear Regression

In [138]:
# #Set up pipeline for scaling continuous variables
# continuous_pipeline_private= Pipeline(steps=[
#     ('ss', StandardScaler())
# ])

# trans_private = ColumnTransformer(transformers=[
#     ('continuous', continuous_pipeline_private, X_train_private.columns),
# ])

In [139]:
# #for public
# continuous_pipeline_public = Pipeline(steps=[
#     ('ss', StandardScaler())
# ])

# trans_public = ColumnTransformer(transformers=[
#     ('continuous', continuous_pipeline_public, X_train_public.columns),
# ])

## Dummy

In [140]:
# #Pipeline for running the model
# dummy_private = Pipeline(steps=[
#     ('trans', trans_private),
#     ('dummy', DummyRegressor())
# ])
# #Fitting and checking the score
# dummy_private.fit(X_train_private, y_train_private)
# dummy_private.score(X_train_private, y_train_private)

0.0

In [141]:
# #Public
# dummy_public = Pipeline(steps=[
#     ('trans', trans_public),
#     ('dummy', DummyRegressor())
# ])
# #Fitting and checking the score
# dummy_public.fit(X_train_public, y_train_public)
# dummy_public.score(X_train_public, y_train_public)

0.0

## Regression

In [142]:
# model_one_private = Pipeline(steps=[
#     ('trans', trans_private),
#     ('ols', LinearRegression())
# ])
# #Fit model on all the data
# model_one_private.fit(X_train_private, y_train_private)
# #Grab predictions and print precision
# y_pred_private = model_one_private.predict(X_train_private)
# print("Training Score:" + str(model_one_private.score(X_train_private, y_train_private)))
# #Run a cross validation to test for overfitting
# scores_private = np.mean(cross_val_score(model_one_private, X_train_private, y_train_private, cv=5))
# print("Validation Score:" + str(scores_private))

Training Score:0.40532715885833936
Validation Score:0.39858985537719926


In [143]:
# model_one_public = Pipeline(steps=[
#     ('trans', trans_public),
#     ('ols', LinearRegression())
# ])
# #Fit model on all the data
# model_one_public.fit(X_train_public, y_train_public)
# #Grab predictions and print precision
# y_pred_private = model_one_public.predict(X_train_public)
# print("Training Score:" + str(model_one_public.score(X_train_public, y_train_public)))
# #Run a cross validation to test for overfitting
# scores_public = np.mean(cross_val_score(model_one_public, X_train_public, y_train_public, cv=5))
# print("Validation Score:" + str(scores_public))

Training Score:0.3930757800870507
Validation Score:0.3771776311179294


In [144]:
# #Feature Importance
# important_private = []
# for name, coef in zip(X_train_private.columns, model_one_private['ols'].coef_):
#     if coef > 0:
#         important_private.append((name, coef))
# important_private

[('F2TUFEFT', 7.798005034822642),
 ('F2ENDMFT', 2.931106805361343),
 ('F2STSVFT', 4.681762265143359),
 ('F2EQUITR', 1.1250560934357974)]

In [153]:
# X_train_private= X_train_private[['F2TUFEFT', 'F2STSVFT', 'F2ENDMFT', 'F2EQUITR']]


,F2TUFEFT,F2STSVFT,F2ENDMFT,F2EQUITR
F2TUFEFT,True,False,False,False
F2STSVFT,False,True,False,False
F2ENDMFT,False,False,True,False
F2EQUITR,False,False,False,True


In [146]:
# #Feature Importance
# important_public = []
# for name, coef in zip(X_train_public.columns, model_one_public['ols'].coef_):
#     if coef > 0:
#         important_public.append((name, coef))

# important_public

[('F1TUFEFT', 8.62416552388888),
 ('F1COREXP', 6.850518023729404),
 ('F1RSRCPC', 4.262615005239424),
 ('F1STAPFT', 2.941290537863461),
 ('F1ACSPPC', 3.8313015190633446),
 ('F1INSTPC', 2.1531776716077253)]

In [155]:
# X_train_public=X_train_public[['F1TUFEFT', 'F1STAPFT', 'F1COREXP', 'F1ACSPPC', 'F1RSRCPC','F1INSTPC']]


,F1TUFEFT,F1STAPFT,F1COREXP,F1ACSPPC,F1RSRCPC,F1INSTPC
F1TUFEFT,True,False,False,False,False,False
F1STAPFT,False,True,False,False,False,False
F1COREXP,False,False,True,False,False,False
F1ACSPPC,False,False,False,True,False,False
F1RSRCPC,False,False,False,False,True,False
F1INSTPC,False,False,False,False,False,True


# Second Model

In [150]:
# model_private = sm.OLS(y_train_private, X_train_private).fit()
# model_private.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               GBA6RTBK   R-squared (uncentered):                   0.833
Model:                            OLS   Adj. R-squared (uncentered):              0.833
Method:                 Least Squares   F-statistic:                              4462.
Date:                Tue, 30 Nov 2021   Prob (F-statistic):                        0.00
Time:                        18:56:09   Log-Likelihood:                         -15987.
No. Observations:                3578   AIC:                                  3.198e+04
Df Residuals:                    3574   BIC:                                  3.201e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
F2TUFEFT       0.0017   4.85e-05     34.233      0.000       0.002       0.002
F2STSVFT       0.0004      0.000      2.824      0.005       0.000       0.001
F2ENDMFT    3.151e-05   1.98e-06     15.881      0.000    2.76e-05    3.54e-05
F2EQUITR       0.1786      0.012     14.440      0.000       0.154       0.203
==============================================================================
Omnibus:                      274.012   Durbin-Watson:                   2.049
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1515.687
Skew:                          -0.056   Prob(JB):                         0.00
Kurtosis:                       6.187   Cond. No.                     7.73e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 7.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [151]:
# model_public = sm.OLS(y_train_public, X_train_public).fit()
# model_public.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               GBA6RTBK   R-squared (uncentered):                   0.865
Model:                            OLS   Adj. R-squared (uncentered):              0.865
Method:                 Least Squares   F-statistic:                              2075.
Date:                Tue, 30 Nov 2021   Prob (F-statistic):                        0.00
Time:                        18:56:12   Log-Likelihood:                         -8194.8
No. Observations:                1943   AIC:                                  1.640e+04
Df Residuals:                    1937   BIC:                                  1.644e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
F1TUFEFT       0.0016      0.000     11.517      0.000       0.001       0.002
F1STAPFT       0.0005   7.17e-05      7.028      0.000       0.000       0.001
F1COREXP    9.227e-09   9.36e-10      9.854      0.000    7.39e-09    1.11e-08
F1ACSPPC       0.2680      0.087      3.070      0.002       0.097       0.439
F1RSRCPC       0.3037      0.051      5.988      0.000       0.204       0.403
F1INSTPC       0.3632      0.026     13.912      0.000       0.312       0.414
==============================================================================
Omnibus:                       54.911   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               81.719
Skew:                           0.279   Prob(JB):                     1.80e-18
Kurtosis:                       3.836   Cond. No.                     1.56e+08
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.56e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""